<a href="https://colab.research.google.com/github/papipopapu/PhysicsThings/blob/main/FluidFuckYou.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import numba as nb
import typing as tp
import math
import time
!pip install --upgrade tbb


In [2]:
# Methods
# Keep in mind we need Gradient(p) to be substracted of non boundary velocities
# , while Diverence(\vec{u}) is to be substracted of pressure

@nb.njit # trimmed
def DY(V: np.array):
  dV = V[2:, 1:-1] - V[:V.shape[0]-2, 1:-1]
  return dV
@nb.njit # trimmed
def DX(V: np.array):
  dV = V[1:-1, 1:-1] - V[:V.shape[0]-2, 1:-1]
  return dV

@nb.njit
def setBounds(S: np.array, bType: int):
  Ni = S.shape[0]; Nj = S.shape[1] 
  if bType is 1:
    S[:, 0] = -S[:, 1];    S[:, Nj-1] = -S[:, Nj-2];    S[0, :] = S[1, :];      S[Ni-1, :] = S[Ni-2, :]; 
  if bType is 2:
    S[:, 0] = S[:, 1];     S[:, Nj-1] = S[:, Nj-2];     S[0, :] = -S[1, :];     S[Ni-1, :] = -S[Ni-2, :]; 
  else:
    S[:, 0] = S[:, 1];     S[:, Nj-1] = S[:, Nj-2];     S[0, :] = S[1, :];      S[Ni-1, :] = S[Ni-2, :]; 
    
@nb.njit
def Diffuse(S1: np.array, S0: np.array, boundType: int, dh: float, dt: float, visc: float):
  a = 4 * dt/(dh*dh) * visc
  Ni = S0.shape[0]; Nj = S0.shape[1] 
  for k in np.arange(20):
    for i in nb.prange(1, Ni + 1):
      for j in np.arange(1, Nj + 1):
        S1[i, j] = (S0[i, j] + a * (S1[i+1, j] + S1[i-1, j] + S1[i, j+1] + S1[i, j-1]))/(1 + 4*a)
    setBounds(S1, boundType)
  
@nb.njit(parallel = True)
def Advect(S1: np.array, S0: np.array, u: np.array, v: np.array, boundType: int, dh: float, dt: float):
  Ni = S0.shape[0]; Nj = S0.shape[1]
  dtp = dt/dh
  for i in nb.prange(1, Ni + 1):
    for j in np.arange(1, Nj + 1):
       jp = j - dtp * u[i, j]; ip = i + dtp * v[i, j] # it makes sense
       if jp < 0.5 : jp = 0.5
       if ip < 0.5 : ip = 0.5
       if jp > Nj + 0.5 : jp = Nj + 0.5
       if ip > Ni + 0.5 : ip = Ni + 0.5;
       iTop = int(ip); jLeft = int(jp); iBottom = int(ip+1); jRight = int(jp+1)
       interpY = (ip-iTop)/(iBottom-iTop); interpX = (jp-jLeft)/(jRight-jLeft)
       SL = S0[iTop, jLeft]  * (1 - interpY) + S0[iBottom, jLeft]  *  interpY
       SR = S0[iTop, jRight] * (1 - interpY) + S0[iBottom, jRight] * interpY
       S1[i, j] =         SL * (1 - interpX) +                  SR * interpX
  setBounds(S1, boundType)




class Fluid:
  def __init__(self, Ni: int, Nj: int, dh: float, u0: np.array, v0: np.array, p0: np.array, s0: np.array):
    self._u0 = u0
    self._u1 = np.zeros((Ni+2, Nj+2))
    self._v0 = v0
    self._v1 = np.zeros((Ni+2, Nj+2))
    self._p0 = p0
    self._p1 = np.zeros((Ni+2, Nj+2))
    self._s0 = s0
    self._s1 = np.zeros((Ni+2, Nj+2))

    self._dt = 1
    self._dh = dh
    self._Ni = Ni
    self._Nj = Nj

  
  def step(self, dt):
    self._dt = dt

    # Vstep:
    # - Add forces
    # - Enforce Boundaries
    # - Diffuse
    # - Enforce Boundaries
    # - Project
    # - Advect
    # - Enforce BOundaries
    # - Project

    # Vstep:
    # - Add sources
    # - Advect
    # - Enforce Boundaries
    # - Diffuse
    # - Enforce Boundaries

  




In [72]:
s0 = np.arange(10000).reshape(100,100)/10000


In [73]:
%%timeit
s1 = np.copy(s0)
Diffuse(s1, s0, 0, 1., 1., 1.)


100 loops, best of 5: 2.44 ms per loop


In [59]:
!pip install --upgrade tbb

In [3]:
v = np.ones((10,10))
u = np.zeros((10,10))
S0 = np.arange(100).reshape(10,10)
S = np.zeros((10, 10))
dt = 1; dh = 1; bty = 0;
S0

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
       [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
       [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
       [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
       [80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
       [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]])

In [6]:
Advect(S, S0, u, v, bty, dh, dt)

In [7]:
Diffuse(S, S0, 0, dh, 1, 1)


In [8]:

S

array([[28.16038748, 28.16038748, 28.55299332, 29.18711349, 29.93403557,
        30.70749441, 31.43620336, 32.04310051, 32.4247449 , 32.4247449 ],
       [28.16038748, 28.16038748, 28.55299332, 29.18711349, 29.93403557,
        30.70749441, 31.43620336, 32.04310051, 32.4247449 , 32.4247449 ],
       [31.79658284, 31.79658284, 32.23610176, 32.89314744, 33.64905965,
        34.42500686, 35.15367043, 35.75990169, 36.14109157, 36.14109157],
       [37.51389893, 37.51389893, 38.07434429, 38.7793618 , 39.55437833,
        40.33750626, 41.06850378, 41.67528629, 42.05653795, 42.05653795],
       [44.39676156, 44.39676156, 45.05114894, 45.79795447, 46.59087385,
        47.38130149, 48.11504258, 48.72273422, 49.10423788, 49.10423788],
       [51.71884769, 51.71884769, 52.41085077, 53.17608477, 53.97752787,
        54.77167022, 55.50689953, 56.11512188, 56.49678729, 56.49678729],
       [58.79495602, 58.79495602, 59.46530724, 60.22151943, 61.01963674,
        61.81263813, 62.54746949, 63.1555391 

In [5]:
!pip install --upgrade tbb




In [159]:
X = np.random.rand(100000) * 100000
Y = np.zeros(100000)

In [160]:
%timeit 10
roundabunch(X, Y)

100000000 loops, best of 5: 12.7 ns per loop


In [111]:
print(X)

[]
